# Softmax Classification
- Softmax
- Cross Entropy
- Low-level Implementation
- High-level Implementation
- Training Example

### Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

In [2]:
# For reproducibility
torch.manual_seed(1)

### Softmax

$ P(class=i) = \frac{e^i}{\sum e^i} $

In [3]:
z = torch.FloatTensor([1, 2, 3])
z

tensor([1., 2., 3.])

In [4]:
hypothesis = F.softmax(z, dim = 0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [5]:
hypothesis.sum()

tensor(1.)

### Cross Entropy Loss (Low-level)

$ Loss = \frac{1}{N} \sum - y \log(\hat{y}) $

In [6]:
z = torch.rand(3, 5, requires_grad = True)
z

tensor([[0.7576, 0.2793, 0.4031, 0.7347, 0.0293],
        [0.7999, 0.3971, 0.7544, 0.5695, 0.4388],
        [0.6387, 0.5247, 0.6826, 0.3051, 0.4635]], requires_grad=True)

In [7]:
hypothesis = F.softmax(z, dim = 1)
print(hypothesis)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward>)


In [8]:
y = torch.randint(5, (3, )).long()
print(y)

tensor([0, 2, 1])


In [9]:
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [10]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim = 1).mean()
print(cost)

tensor(1.4689, grad_fn=<MeanBackward0>)


### Cross-entropy Loss with torch.nn.functional

In [11]:
# Low level
torch.log(F.softmax(z, dim=1))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward>)

In [12]:
# High level
F.log_softmax(z, dim=1)

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward>)

In [13]:
# Low level
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

tensor(1.4689, grad_fn=<MeanBackward0>)

In [14]:
# High level
F.nll_loss(F.log_softmax(z, dim=1), y)

tensor(1.4689, grad_fn=<NllLossBackward>)

In [15]:
F.cross_entropy(z, y) # merge F.log_softmax() and F.nll_loss()

tensor(1.4689, grad_fn=<NllLossBackward>)

### Training with Low-level Cross Entropy Loss

In [16]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]

y_train = [2, 2, 2, 1, 1, 1, 0, 0] # class 0, 1, 2

In [17]:
x_train = torch.FloatTensor(x_train)
x_train.shape

torch.Size([8, 4])

In [18]:
y_train = torch.LongTensor(y_train)
y_train.shape

torch.Size([8])

In [19]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr = 0.1)

epochs = 1000
for epoch in range(epochs + 1):
    # Cost 계산 (1)
    hypothesis = F.softmax(x_train.matmul(W) + b, dim = 1)
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim = 1))).sum(dim = 1).mean()

    # Cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        pred = hypothesis.argmax(dim = 1)
        corrects_pred = (pred == y_train)
        accuracy = corrects_pred.sum().float() / float(y_one_hot.size(0))
        print('Epoch {:4d}/{} Accuracy: {:2.2f}% Cost: {:.6f}'.format(
            epoch, epochs, accuracy, cost.item()
        ))

Epoch    0/1000 Accuracy: 0.25% Cost: 1.098612
Epoch  100/1000 Accuracy: 0.88% Cost: 0.901535
Epoch  200/1000 Accuracy: 0.88% Cost: 0.839114
Epoch  300/1000 Accuracy: 0.88% Cost: 0.807826
Epoch  400/1000 Accuracy: 0.88% Cost: 0.788472
Epoch  500/1000 Accuracy: 0.88% Cost: 0.774822
Epoch  600/1000 Accuracy: 0.88% Cost: 0.764449
Epoch  700/1000 Accuracy: 0.88% Cost: 0.756191
Epoch  800/1000 Accuracy: 0.88% Cost: 0.749398
Epoch  900/1000 Accuracy: 0.88% Cost: 0.743671
Epoch 1000/1000 Accuracy: 0.88% Cost: 0.738749


### High-level Implementation with nn.Module

In [20]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]

y_train = [2, 2, 2, 1, 1, 1, 0, 0] # class 0, 1, 2

In [21]:
x_train = torch.FloatTensor(x_train)
x_train.shape

torch.Size([8, 4])

In [22]:
y_train = torch.LongTensor(y_train)
y_train.shape

torch.Size([8])

In [23]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3)

    def forward(self, x):
        return self.linear(x)

In [24]:
model = SoftmaxClassifierModel()

In [25]:
optimizer = optim.SGD(model.parameters(), lr = 0.1)

epochs = 1000
for epoch in range(epochs + 1):
    # H(x) 계산
    pred = model(x_train)

    # Cost 계산
    cost = F.cross_entropy(pred, y_train)

    # Cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        pred = pred.argmax(dim = 1)
        corrects_pred = (pred == y_train)
        accuracy = corrects_pred.sum().float() / float(y_train.size(0))
        print("Epoch {:4d}/{} Accuracy: {:2.2f}% Cost: {:.6f}".format(
            epoch, epochs, accuracy, cost.item()
        ))

Epoch    0/1000 Accuracy: 0.38% Cost: 1.849513
Epoch  100/1000 Accuracy: 0.75% Cost: 0.689894
Epoch  200/1000 Accuracy: 0.75% Cost: 0.609259
Epoch  300/1000 Accuracy: 0.75% Cost: 0.551218
Epoch  400/1000 Accuracy: 0.88% Cost: 0.500141
Epoch  500/1000 Accuracy: 0.88% Cost: 0.451947
Epoch  600/1000 Accuracy: 0.88% Cost: 0.405051
Epoch  700/1000 Accuracy: 0.88% Cost: 0.358733
Epoch  800/1000 Accuracy: 0.88% Cost: 0.312912
Epoch  900/1000 Accuracy: 0.88% Cost: 0.269522
Epoch 1000/1000 Accuracy: 1.00% Cost: 0.241922


# Fancy Softmax Classification

### Imports

In [26]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

In [27]:
# For reproducibility
torch.manual_seed(1)

### Cross-entropy Loss with torch.nn.functional

In [28]:
z = torch.rand(3, 5, requires_grad = True)
z

tensor([[0.7576, 0.2793, 0.4031, 0.7347, 0.0293],
        [0.7999, 0.3971, 0.7544, 0.5695, 0.4388],
        [0.6387, 0.5247, 0.6826, 0.3051, 0.4635]], requires_grad=True)

In [29]:
hypothesis = F.softmax(z, dim = 1)
hypothesis

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward>)

In [30]:
y = torch.randint(5, (3,)).long()
y

tensor([0, 2, 1])

In [31]:
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

PyTorch has F.log_softmax() function.

In [32]:
# Low level
torch.log(F.softmax(z, dim=1))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward>)

In [33]:
# High level
F.log_softmax(z, dim=1)

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward>)

PyTorch also has F.nll_loss() function that computes the negative loss likelihood.

In [34]:
# Low level
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

tensor(1.4689, grad_fn=<MeanBackward0>)

In [35]:
# High level
F.nll_loss(F.log_softmax(z, dim=1), y.long())

tensor(1.4689, grad_fn=<NllLossBackward>)

## load data

### Data

In [36]:
!git clone https://github.com/yeonwoo780/deeplearning_basic.git

fatal: destination path 'deeplearning_basic' already exists and is not an empty directory.


In [37]:
!ls -lia
!ls -lia deeplearning_basic
!ls -lia deeplearning_basic/Pytorch_dataset

total 20
6422547 drwxr-xr-x 1 root root 4096 Jun  6 10:45 .
6291531 drwxr-xr-x 1 root root 4096 Jun  6 10:43 ..
5111823 drwxr-xr-x 4 root root 4096 Jun  1 13:40 .config
6291536 drwxr-xr-x 7 root root 4096 Jun  6 10:45 deeplearning_basic
6422548 drwxr-xr-x 1 root root 4096 Jun  1 13:40 sample_data
total 40
6291536 drwxr-xr-x 7 root root 4096 Jun  6 10:45 .
6422547 drwxr-xr-x 1 root root 4096 Jun  6 10:45 ..
6291624 drwxr-xr-x 5 root root 4096 Jun  6 10:45 Chapter10~19
6291687 drwxr-xr-x 2 root root 4096 Jun  6 10:45 Chapter1~9
6291703 drwxr-xr-x 2 root root 4096 Jun  6 10:45 dataset
6291537 drwxr-xr-x 8 root root 4096 Jun  6 10:45 .git
6291608 -rw-r--r-- 1 root root   13 Jun  6 10:45 .gitignore
6291697 drwxr-xr-x 2 root root 4096 Jun  6 10:45 Pytorch_dataset
6291710 -rw-r--r-- 1 root root  392 Jun  6 10:45 pyvenv.cfg
6291702 -rw-r--r-- 1 root root 3774 Jun  6 10:45 README.md
total 112
6291697 drwxr-xr-x 2 root root  4096 Jun  6 10:45 .
6291536 drwxr-xr-x 7 root root  4096 Jun  6 10:45 .

In [38]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os

In [39]:
xy = np.loadtxt('/content/deeplearning_basic/Pytorch_dataset/data-04-zoo.csv',
                delimiter = ',', dtype = np.float32)

In [40]:
x_train = torch.FloatTensor(xy[:, 0:-1])
y_train = torch.LongTensor(xy[:, [-1]]).squeeze()

In [41]:
print(x_train.shape) # x_train shape
print(len(x_train))  # x_train 길이
print(x_train[:5])   # 첫 다섯 개

torch.Size([101, 16])
101
tensor([[1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 4., 0., 0., 1.],
        [1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 4., 1., 0., 1.],
        [0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0.],
        [1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 4., 0., 0., 1.],
        [1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 4., 1., 0., 1.]])


In [42]:
print(y_train.shape) # y_train shape
print(len(y_train))  # y_train 길이
print(y_train[:5])   # 첫 다섯 개

torch.Size([101])
101
tensor([0, 0, 3, 0, 0])


In [43]:
nb_classes = 7
y_one_hot = torch.zeros((len(y_train), nb_classes))
y_one_hot = y_one_hot.scatter(1, y_train.unsqueeze(1), 1)

In [44]:
y_one_hot[:5]

tensor([[1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.]])

In [45]:
# 모델 초기화
W = torch.zeros((x_train.shape[1], y_one_hot.shape[1]), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr = 0.1)

epochs = 1000
for epoch in range(epochs + 1):
    pred = x_train.matmul(W) + b
    cost = F.cross_entropy(pred, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        pred = pred.argmax(dim = 1)
        corrects_pred = (pred == y_train)
        accuracy = corrects_pred.sum().float() / float(y_one_hot.size(0))
        print('Epoch {:4d}/{} Accuracy: {:2.2f}% Cost: {:.6f}'.format(
            epoch, epochs, accuracy * 100, cost.item()
        ))


Epoch    0/1000 Accuracy: 40.59% Cost: 1.945909
Epoch  100/1000 Accuracy: 87.13% Cost: 0.471836
Epoch  200/1000 Accuracy: 93.07% Cost: 0.326327
Epoch  300/1000 Accuracy: 95.05% Cost: 0.257839
Epoch  400/1000 Accuracy: 95.05% Cost: 0.215762
Epoch  500/1000 Accuracy: 95.05% Cost: 0.186603
Epoch  600/1000 Accuracy: 97.03% Cost: 0.164898
Epoch  700/1000 Accuracy: 98.02% Cost: 0.147955
Epoch  800/1000 Accuracy: 100.00% Cost: 0.134278
Epoch  900/1000 Accuracy: 100.00% Cost: 0.122962
Epoch 1000/1000 Accuracy: 100.00% Cost: 0.113422


### High-level Implementation with nn.Module

In [46]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(16, 7)
    def forward(self, x):
        return self.linear(x)

In [47]:
model = SoftmaxClassifierModel()

In [48]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    pred= model(x_train)

    # cost 계산
    cost = F.cross_entropy(pred, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    if epoch % 100 == 0:
        pred = pred.argmax(dim = 1)
        corrects_pred = (pred == y_train)
        accuracy = corrects_pred.sum().float() / float(y_train.size(0))
        print('Epoch {:4d}/{} Accuracy: {:2.2f}% Cost: {:.6f}'.format(
            epoch, epochs, accuracy * 100, cost.item()
        ))

Epoch    0/1000 Accuracy: 9.90% Cost: 1.919160
Epoch  100/1000 Accuracy: 88.12% Cost: 0.468405
Epoch  200/1000 Accuracy: 92.08% Cost: 0.320585
Epoch  300/1000 Accuracy: 95.05% Cost: 0.248953
Epoch  400/1000 Accuracy: 95.05% Cost: 0.204819
Epoch  500/1000 Accuracy: 97.03% Cost: 0.174506
Epoch  600/1000 Accuracy: 97.03% Cost: 0.152248
Epoch  700/1000 Accuracy: 100.00% Cost: 0.135139
Epoch  800/1000 Accuracy: 100.00% Cost: 0.121543
Epoch  900/1000 Accuracy: 100.00% Cost: 0.110461
Epoch 1000/1000 Accuracy: 100.00% Cost: 0.101245
